In [1]:
import os
import re
import csv
import xml.etree.ElementTree as ET
from collections import defaultdict

# https://www.fasb.org/page/detail?pageId=/projects/FASB-Taxonomies/2025-gaap-financial-reporting-taxonomy.html
# 2025 Taxonomy downloads: https://xbrl.fasb.org/us-gaap/2025/us-gaap-2025.zip

# === CONFIG ===
TAXONOMY_DIR = "data/us-gaap-2025/"
ELTS_XSD = os.path.join(TAXONOMY_DIR, "elts", "us-gaap-2025.xsd")
STM_DIR = os.path.join(TAXONOMY_DIR, "stm")

OUTPUT_PATH = "data/WITH_TAXONOMY_HIERARCHY_us_gaap_2025_with_all_statements_and_hierarchy.csv"

FILENAME_STATEMENT_MAP = {
    "scf": "Cash Flow Statement",
    "soi": "Income Statement",
    "sfp": "Balance Sheet",
    "sheci": "Equity Statement",
    "soc": "Comprehensive Income"
}

XBRLI_NS = "http://www.xbrl.org/2003/instance"
BALANCE_KEY = f"{{{XBRLI_NS}}}balance"
PERIOD_TYPE_KEY = f"{{{XBRLI_NS}}}periodType"


def generate_description(tag_name):
    return re.sub(
        r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])',
        ' ',
        tag_name
    ).lower()


# === STEP 1: TAG METADATA ===
tag_metadata = {}
tree = ET.parse(ELTS_XSD)
root = tree.getroot()
for el in root.findall(".//{http://www.w3.org/2001/XMLSchema}element"):
    name = el.attrib.get("name")
    if not name or el.attrib.get("abstract") == "true":
        continue
    tag_metadata[name] = {
        "balance": el.attrib.get(BALANCE_KEY, ""),
        "period_type": el.attrib.get(PERIOD_TYPE_KEY, ""),
        "statements": set()
    }

# === STEP 2: STATEMENT TYPE MAPPING ===
for file in os.listdir(STM_DIR):
    if not file.endswith(".xml") or "-pre-" not in file:
        continue
    parts = file.split("-")
    if len(parts) < 4:
        continue
    stmt_key = parts[3]
    inferred_statement = FILENAME_STATEMENT_MAP.get(stmt_key)
    if not inferred_statement:
        continue

    tree = ET.parse(os.path.join(STM_DIR, file))
    root = tree.getroot()
    for loc in root.findall(".//{http://www.xbrl.org/2003/linkbase}loc"):
        href = loc.attrib.get("{http://www.w3.org/1999/xlink}href", "")
        tag = href.split("#")[-1]
        if tag.startswith("us-gaap_"):
            tag = tag.replace("us-gaap_", "")
        if tag in tag_metadata:
            tag_metadata[tag]["statements"].add(inferred_statement)

# === STEP 3: BUILD BASE OUTPUT ===
final_output = []
for tag, meta in tag_metadata.items():
    for stmt in meta["statements"]:
        final_output.append({
            "tag": tag,
            "statement_type": stmt,
            "balance": meta["balance"],
            "period_type": meta["period_type"],
            "description": generate_description(tag),
            "subcategory_path": ""
        })

# === STEP 4: EXTRACT TAXONOMY HIERARCHY PATHS ===
def extract_tag_hierarchy_paths(pre_files):
    tag_paths = defaultdict(set)

    for path in pre_files:
        try:
            tree = ET.parse(path)
            root = tree.getroot()

            loc_map = {}
            arcs_by_from = defaultdict(list)

            for loc in root.findall(".//{http://www.xbrl.org/2003/linkbase}loc"):
                loc_id = loc.attrib.get("{http://www.w3.org/1999/xlink}label")
                href = loc.attrib.get("{http://www.w3.org/1999/xlink}href", "")
                if loc_id and "us-gaap_" in href:
                    tag = href.split("#")[-1]
                    loc_map[loc_id] = tag

            for arc in root.findall(".//{http://www.xbrl.org/2003/linkbase}presentationArc"):
                from_id = arc.attrib.get("{http://www.w3.org/1999/xlink}from")
                to_id = arc.attrib.get("{http://www.w3.org/1999/xlink}to")
                arcs_by_from[from_id].append(to_id)

            all_from = set(arcs_by_from.keys())
            all_to = {to for children in arcs_by_from.values() for to in children}
            root_ids = all_from - all_to

            def walk(current_id, path):
                tag = loc_map.get(current_id)
                current_path = path + [tag] if tag else path
                for child_id in arcs_by_from.get(current_id, []):
                    child_tag = loc_map.get(child_id)
                    if child_tag and child_tag.startswith("us-gaap_"):
                        clean_tag = child_tag.replace("us-gaap_", "")
                        tag_paths[clean_tag].add(" > ".join(current_path + [clean_tag]))
                    walk(child_id, current_path)

            for root_id in root_ids:
                walk(root_id, [])

        except ET.ParseError:
            continue

    return tag_paths


# Gather all presentation files (stm + dis + others)
pre_files = []
for root, _, files in os.walk(TAXONOMY_DIR):
    for file in files:
        if "-pre-" in file and file.endswith(".xml"):
            pre_files.append(os.path.join(root, file))

tag_hierarchy = extract_tag_hierarchy_paths(pre_files)

# === STEP 5: ATTACH HIERARCHY TO TAGS ===
ALL_STATEMENTS = {
    "Cash Flow Statement",
    "Income Statement",
    "Balance Sheet",
    "Equity Statement",
    "Comprehensive Income"
}

final_filtered = []
for row in final_output:
    if row["statement_type"] in ALL_STATEMENTS:
        tag = row["tag"]
        row["taxonomy_hierarchy"] = (
            list(tag_hierarchy.get(tag))[0] if tag in tag_hierarchy else ""
        )
        final_filtered.append(row)

# === STEP 6: WRITE OUTPUT CSV ===
# os.makedirs("output", exist_ok=True)
with open(OUTPUT_PATH, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=[
        "tag", "statement_type", "balance", "period_type",
        "description", "subcategory_path", "taxonomy_hierarchy"
    ])
    writer.writeheader()
    writer.writerows(final_filtered)


# Append OFSS IDs

In [ ]:
import csv
import json
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# === CONFIG ===
MODEL_NAME = "BAAI/bge-large-en-v1.5"
INPUT_CSV = "data/WITH_TAXONOMY_HIERARCHY_us_gaap_2025_with_all_statements_and_hierarchy.csv"
OFSS_JSON = "../shared/open_financial_statement_schema.json"
OUTPUT_CSV = "data/with_ofss_ids.csv"
SIMILARITY_THRESHOLD = 0.7

# === DEVICE SETUP ===
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = SentenceTransformer(MODEL_NAME, device=device)

# === FLATTEN NESTED OFSS MAP ===
def flatten_ofss(d, parent_key=""):
    flat = {}
    for k, v in d.items():
        new_key = f"{parent_key}/{k}" if parent_key else k
        if isinstance(v, dict):
            flat.update(flatten_ofss(v, new_key))
        else:
            flat[new_key] = v
    return flat

def normalize_stmt_prefix(stmt):
    stmt_prefix = stmt.lower().replace(" statement", "")
    return stmt_prefix

# Enhanced validation rules
def is_valid_mapping(stmt, balance, tag, mapped_name):
    stmt = stmt.lower()
    balance = str(balance).lower()
    mapped = mapped_name.lower()
    tag = tag.lower()

    # === Balance Sheet Validation ===
    # Prevent debit items mapping to liabilities/equity accounts
    if stmt == "balance sheet":
        if balance == "debit" and any(x in mapped for x in [
            "liabilities", "payable", "accrued", "common stock"
        ]):
            return False

        # Prevent credit items mapping to asset accounts
        if balance == "credit" and any(x in mapped for x in [
            "assets", "cash", "inventory", "receivable"
        ]):
            return False

        # Commercial paper should not be classified as shareholders' equity
        if "commercial paper" in tag and "shareholders' equity" in mapped:
            return False

        # Allowances should not map to liabilities
        if "allowance" in tag and "liabilities" in mapped:
            return False

        # Treasury stock should only map to equity contexts
        if "treasury stock" in tag and not (
            "shareholders' equity" in mapped or "equity statement" in mapped
        ):
            return False
        
        # Treasury stock must not be classified as assets or cash
        if "treasury stock" in tag and any(x in mapped for x in ["asset", "cash"]):
            return False
        
        # Restrictions on treasury stock should not be mapped to asset classes
        if "restriction" in tag and "treasury stock" in tag and any(
            x in mapped for x in ["cash", "asset", "receivable"]
        ):
            return False

        # Collateral should not map to goodwill (unrelated asset types)
        if "collateral" in tag and "goodwill" in mapped:
            return False

        # Derivative contracts should not map to minority interest
        if "derivatives and other contracts" in tag and "minority interest" in mapped:
            return False

        # Customer contracts should not map to property
        if "contract with customer" in tag and "property" in mapped:
            return False
        
        # Balance Sheet: retail deposits ≠ customer advances
        if "deposits retail" in tag and "customer advances" in mapped:
            return False

        # Convertible notes must retain convertible classification
        if "convertible" in tag and "convertible" not in mapped:
            return False
        
        # Discontinued operation assets ≠ generic 'other current assets'
        if "discontinued operation" in tag and "other current assets" in mapped:
            return False
        
        # Assets held for sale must not be grouped under 'other current assets'
        if "assets held for sale" in tag and "other current assets" in mapped:
            return False
        
        # Assets held for sale must not be grouped under 'other current assets'
        if "assets held for sale" in tag and "other current assets" in mapped:
            return False

        # Cumulative dividends ≠ accumulated other comprehensive income
        if "cumulative dividends" in tag and "other comprehensive income" in mapped:
            return False

        # Bank-owned life insurance ≠ other long term assets
        if "bank owned life insurance" in tag and "long term assets" in mapped:
            return False

        # Allowance for loans receivable must not map to empty or unrelated areas
        if "allowance" in tag and "receivable" in tag and mapped.strip() == "":
            return False

        # Construction loans ≠ other current liabilities
        if "construction loan" in tag and "other current liabilities" in mapped:
            return False

        # Market risk benefit ≠ equity accounts
        if "market risk benefit" in tag and any(x in mapped for x in ["equity", "common stock"]):
            return False
        
        # Loan receivable allowance must not map to interest expense
        if "allowance" in tag and "loan" in tag and "interest expense" in mapped:
            return False

        # Deferred income tax asset should not map to cash or equivalents
        if "deferred tax asset" in tag and "cash" in mapped:
            return False

        # Preferred equity ≠ shareholders' equity total
        if "preferred equity" in tag and "shareholders' equity" in mapped and "total" in mapped:
            return False

        # Pension asset ≠ other current assets
        if "pension" in tag and "asset" in tag and "other current assets" in mapped:
            return False

        # Advances to suppliers should not map to inventories
        if "advance to supplier" in tag and "inventory" in mapped:
            return False

        # Advances to suppliers ≠ inventory
        if "advance to supplier" in tag and "inventory" in mapped:
            return False

        # Deferred revenue ≠ equity
        if "deferred revenue" in tag and "equity" in mapped:
            return False

        # Investment tax credit ≠ cash
        if "investment tax credit" in tag and "cash" in mapped:
            return False

        # Allowance for doubtful accounts ≠ other current assets
        if "allowance" in tag and "doubtful" in tag and "other current assets" in mapped:
            return False

        # Lease asset should not map to deferred revenue
        if "lease" in tag and "asset" in tag and "deferred revenue" in mapped:
            return False

        # Treasury stock assets must not map to intangible assets
        if "treasury stock" in tag and "intangible" in mapped:
            return False

        # Deferred tax liabilities ≠ equity
        if "deferred tax liability" in tag and "equity" in mapped:
            return False

        # Noncontrolling interest ≠ accumulated other comprehensive income
        if "noncontrolling interest" in tag and "comprehensive income" in mapped:
            return False

        # Advances to affiliates ≠ inventory
        if "advance" in tag and "affiliate" in tag and "inventory" in mapped:
            return False

        # Receivables-related allowances must not map to cash or other current assets
        if "allowance" in tag and any(x in mapped for x in ["cash", "other current assets"]):
            return False

        # Lease receivable ≠ other long term assets
        if "lease receivable" in tag and "other long term assets" in mapped:
            return False

        # Advances to related parties ≠ inventories
        if "advance" in tag and "related party" in tag and "inventory" in mapped:
            return False

        # Intangibles with definite life ≠ goodwill
        if "definite lived" in tag and "goodwill" in mapped:
            return False




    # === Cash Flow Statement Validation ===
    if stmt == "cash flow statement":
        investing_keywords = [
            "acquire", "investment", "purchase", "building", "property",
            "fund", "financing", "debt", "deposit", "debt reduction",
            "proceeds", "advance", "contribution"
        ]
        financing_keywords = ["distribution", "dividend", "redeemable securities"]

        # Block investing or financing terms mapping to operating activities
        if any(keyword in tag for keyword in investing_keywords) and "operating activities" in mapped:
            return False
        if any(keyword in tag for keyword in financing_keywords) and "operating activities" in mapped:
            return False

        # Subordinated debt repayment should not map to debt issuance
        if "repayments of subordinated debt" in tag and "debt issued" in mapped:
            return False

        # Date-related metadata should not map to cash flows
        if "original debt issuance date" in tag and "debt issued" in mapped:
            return False

        # Deposits should not be mapped to interest income
        if "deposit" in tag and "interest" in mapped:
            return False

        # Advances and contributions should not map to interest income
        if ("advance" in tag or "contribution" in tag) and "interest" in mapped:
            return False

        # Unrealized gain/loss should not map to investment purchases
        if "gainloss" in tag and "purchase of investments" in mapped:
            return False

        # Depreciation should not relate to discontinued ops or regulatory assets
        if "discontinued operations" in tag and "depreciation" in mapped:
            return False
        if "regulatory assets and liabilities" in tag and "depreciation" in mapped:
            return False

        # Dividend-related receivables should not map to interest
        if "dividends receivable" in tag and "interest" in mapped:
            return False

        # Withdrawals from contract holders should not be interest
        if "withdrawal from contract holders" in tag and "interest" in mapped:
            return False

        # Proceeds from secured credit lines ≠ short-term debt issuance
        if "secured lines of credit" in tag and "short term debt issued" in mapped:
            return False

        # Postemployment benefit payments ≠ tax payments
        if "postemployment benefits" in tag and "taxes paid" in mapped:
            return False

        # Specific: annuity/investment repayments ≠ debt issuance
        if "repayments of annuities and investment certificates" in tag and "debt issued" in mapped:
            return False

        # General: any repayment tag should not map to debt issuance
        if "repayment" in tag and "debt issued" in mapped:
            return False
        
        # Lease acquisition costs ≠ business acquisitions
        if "lease acquisition" in tag and "acquisition of business" in mapped:
            return False
        
        # Specific short-term debt repayments ≠ net short-term debt changes
        if "repayment" in tag and "short term debt, net" in mapped:
            return False

        # Deferred purchase price ≠ short-term debt issuance
        if "deferred purchase price" in tag and "short term debt issued" in mapped:
            return False

        # Deferred purchase price ≠ short-term debt issuance
        if "deferred purchase price" in tag and "short term debt issued" in mapped:
            return False

        # Real estate deposits ≠ acquisition of business
        if "real estate" in tag and "deposits" in tag and "acquisition of business" in mapped:
            return False

        # Repayments on lease ≠ equity issuance
        if "lease" in tag and "proceeds" in tag and "issuance of common" in mapped:
            return False

        # Notes receivable ≠ accounts receivable
        if "notes receivable" in tag and "accounts receivable" in mapped:
            return False

        # Loan issuance ≠ business acquisition
        if "loan" in tag and "issuance" in tag and "acquisition of business" in mapped:
            return False

        # Contribution of property ≠ investing outflows
        if "contribution" in tag and "property" in tag and "investing" in mapped:
            return False

        # Payment of legal settlement ≠ tax payment
        if "legal settlement" in tag and "taxes paid" in mapped:
            return False

        # Purchase of insurance policies ≠ investment securities
        if "insurance" in tag and "purchase" in tag and "investment securities" in mapped:
            return False

        # Business acquisition ≠ repayment of debt
        if "acquisition" in tag and "repayment" in mapped:
            return False

        # Purchase of insurance ≠ investment purchase
        if "insurance" in tag and "purchase" in tag and "investment" in mapped:
            return False

        # Real estate construction deposits ≠ acquisition of business
        if "deposit" in tag and "construction" in tag and "real estate" in tag and "acquisition of business" in mapped:
            return False

        # Payment of preferred dividends ≠ issuance of debt
        if "dividend" in tag and "preferred" in tag and "debt issued" in mapped:
            return False

        # Purchase of time deposits ≠ property acquisition
        if "time deposit" in tag and "property" in mapped:
            return False

        # Insurance settlement ≠ investing activities
        if "insurance settlement" in tag and "investing" in mapped:
            return False

        # Repurchase of convertible debt ≠ common stock issuance
        if "convertible" in tag and "repurchase" in tag and "issuance of common" in mapped:
            return False

        # Loans to affiliates ≠ purchase of equity
        if "loan" in tag and "affiliate" in tag and "purchase of equity" in mapped:
            return False

        # Construction-related deposits ≠ business acquisitions
        if "deposit" in tag and "construction" in tag and "acquisition of business" in mapped:
            return False

        # Purchase of insurance policies ≠ investing cash outflows for securities
        if "insurance" in tag and "purchase" in tag and "investment" in mapped:
            return False

        # Business combination tax adjustments ≠ acquisition cash flow
        if "tax" in tag and "business combination" in tag and "acquisition" in mapped:
            return False

        # Contributions to employee stock plans ≠ equity issuance
        if "employee stock" in tag and "contribution" in tag and "issuance of common" in mapped:
            return False




    # === Comprehensive Income Validation ===
    # Must map to a comprehensive income context
    if stmt == "comprehensive income":
        if "comprehensive income" not in mapped:
            return False

    # === Equity Statement Validation ===
    # Must map to an equity-related section
    if stmt == "equity statement":
        if not ("equity statement" in mapped or "shareholders' equity" in mapped):
            return False
        
        # Equity Statement: total equity incl. NCI ≠ redeemable noncontrolling interest
        if "stockholders equity including portion attributable to noncontrolling" in tag and "redeemable noncontrolling interest" in mapped:
            return False

    # === Income Statement Validation ===
    # Lease income should relate to interest or lease concepts
    if stmt == "income statement":
        if "lease income" in tag and "interest" not in mapped and "lease" not in mapped:
            return False

        # Income from continuing ops should not be tied to minority interest
        if "income loss from continuing operations before income taxes" in tag and "minority interest" in mapped:
            return False

        # Interest/dividend income should not map to preferred dividends
        if "interest and dividend income" in tag and "preferred dividends" in mapped:
            return False

        # Bank-owned life insurance ≠ net sales
        if "bank owned life insurance" in tag and "net sales" in mapped:
            return False

        # Interest expense should not map to any kind of interest income
        if "interest expense" in tag and "interest income" in mapped:
            return False

        # Nonrecurring income should not be classified as investment income
        if "nonrecurring income" in tag and "investment income" in mapped:
            return False

        # Asset-related income ≠ investment income
        if "asset related income" in tag and "investment income" in mapped:
            return False

        # Noncontrolling interest should not be mapped to interest expense
        if "noncontrolling interest" in tag and "interest expense" in mapped:
            return False
        
        # Consumer loan interest income ≠ investment income
        if ("interest and fee income" in tag and "loan" in tag and
            "investment income" in mapped):
            return False
        
        # Production taxes should not be classified under ambiguous 'other' expenses
        if "production tax" in tag and "other operating expenses" in mapped:
            return False
        
        # Variable lease income ≠ net income
        if "lease income" in tag and "net income" in mapped:
            return False
        
        # Variable lease income ≠ net income
        if "lease income" in tag and "net income" in mapped:
            return False

        # Repo interest income ≠ investment income
        if "securities purchased under agreements to resell" in tag and "investment income" in mapped:
            return False

        # Lease-related loan interest ≠ investment income
        if "lease financing" in tag and "investment income" in mapped:
            return False

        # Preferred dividends ≠ interest and dividend income
        if "preferred dividends" in mapped and "interest and dividend income" in tag:
            return False

        # Continuing operations income ≠ minority interest
        if "continuing operations" in tag and "minority interest" in mapped:
            return False

        # Sale of assets ≠ issuance of common stock
        if "sale of" in tag and "issuance of common" in mapped:
            return False

        # Non-interest income ≠ non-operating income
        if "noninterest income" in tag and "non-operating" in mapped:
            return False

        # Marketing and advertising ≠ cost of revenue
        if "advertising" in tag and "cost of revenue" in mapped:
            return False
        
        # Preferred dividends should not map to net sales
        if "preferred dividends" in tag and "net sales" in mapped:
            return False

        # Lease termination gain ≠ cost of revenue
        if "lease termination" in tag and "cost of revenue" in mapped:
            return False

        # Insurance proceeds ≠ other operating income
        if "insurance proceeds" in tag and "operating income" in mapped:
            return False

        # Bank charges ≠ non-operating expense
        if "bank charge" in tag and "non-operating" in mapped:
            return False

        # Restructuring expenses ≠ depreciation or amortization
        if "restructuring" in tag and any(x in mapped for x in ["depreciation", "amortization"]):
            return False

        # Other income ≠ minority interest
        if "other income" in tag and "minority interest" in mapped:
            return False

        # Interest expense ≠ preferred dividends
        if "interest expense" in tag and "preferred dividends" in mapped:
            return False

        # Cost of services ≠ general and admin
        if "cost of services" in tag and "general" in mapped and "admin" in mapped:
            return False

        # Gain on debt extinguishment ≠ minority interest
        if "extinguishment of debt" in tag and "minority interest" in mapped:
            return False

        # Interest income ≠ preferred dividends
        if "interest income" in tag and "preferred dividends" in mapped:
            return False

        # Other comprehensive income ≠ operating revenue
        if "other comprehensive income" in tag and "operating" in mapped:
            return False

        # Tax credit income ≠ investment income
        if "tax credit" in tag and "investment income" in mapped:
            return False

        # Non-interest income ≠ cost of revenue
        if "noninterest" in tag and "cost of revenue" in mapped:
            return False

        # Legal expense ≠ general and admin if specific
        if "legal expense" in tag and "general" in mapped and "admin" in mapped and "settlement" not in tag:
            return False

        # Interest and dividends on bank balances ≠ investment income
        if "interest" in tag and "bank" in tag and "investment income" in mapped:
            return False

        # Net periodic pension cost ≠ net income
        if "pension cost" in tag and "net income" in mapped:
            return False

        # Gain on sale of discontinued ops ≠ other operating income
        if "discontinued" in tag and "gain" in tag and "operating income" in mapped:
            return False

        # Other regulatory adjustments ≠ income from operations
        if "regulatory" in tag and "adjustment" in tag and "income from operations" in mapped:
            return False

        # Debt extinguishment gain/loss ≠ operating income
        if "extinguishment of debt" in tag and "operating income" in mapped:
            return False

        # Income from joint ventures ≠ interest income
        if "joint venture" in tag and "interest income" in mapped:
            return False

        # Other regulatory adjustments ≠ income from operations
        if "regulatory" in tag and "adjustment" in tag and "operating income" in mapped:
            return False



    return True

with open(OFSS_JSON, "r") as f:
    nested_ofss = json.load(f)

flat_ofss = flatten_ofss(nested_ofss)
ofss_names = list(flat_ofss.keys())
ofss_ids = [flat_ofss[name] for name in ofss_names]
ofss_embeddings = model.encode([s.lower() for s in ofss_names], convert_to_tensor=True)

# === STATEMENT NORMALIZATION ===
# STATEMENT_NORMALIZATION = {
#     "Equity Statement": "Balance Sheet",
#     "Comprehensive Income": "Income Statement"
# }

def clean_path(raw):
    tokens = raw.split(" > ")
    tokens = [t for t in tokens if not t.lower().endswith("abstract")]
    tokens = [t.replace("us-gaap_", "").replace("_", " ") for t in tokens]
    return " > ".join(tokens)

# === MATCH FUNCTION ===
def try_match(query_str, stmt, balance, tag):
    query_emb = model.encode(query_str.lower(), convert_to_tensor=True)
    allowed_indices = [
        i for i, name in enumerate(ofss_names)
        if normalize_stmt_prefix(name.lower().split("/")[0]) == normalize_stmt_prefix(stmt)
    ]
    if not allowed_indices:
        return None, None, None

    filtered_embeddings = ofss_embeddings[allowed_indices]
    cos_scores = util.cos_sim(query_emb, filtered_embeddings)[0]

    # Rank all candidates by similarity
    ranked_candidates = sorted(
        zip(allowed_indices, cos_scores),
        key=lambda x: x[1],
        reverse=True
    )

    # Check each candidate explicitly for accounting validity
    for candidate_idx, candidate_score in ranked_candidates:
        if candidate_score < SIMILARITY_THRESHOLD:
            break  # All remaining candidates will be below threshold

        mapped_name = ofss_names[candidate_idx].lower().replace("/", " > ")

        # Explicit validation check
        if is_valid_mapping(stmt, balance, tag, mapped_name):
            return candidate_idx, candidate_score.item(), mapped_name

    # If no valid candidates found
    return None, None, None


# === PROCESS CSV ===
output_rows = []
with open(INPUT_CSV, newline="") as f:
    reader = csv.DictReader(f)
    for row in tqdm(reader, desc="Processing rows"):
        stmt = row["statement_type"].strip()
        # stmt = STATEMENT_NORMALIZATION.get(stmt, stmt)

        tag = row["tag"]
        balance = row["balance"]
        desc = row.get("description", "").strip()
        path_raw = row["taxonomy_hierarchy"].strip()

        row["ofss_id"] = ""
        row["ofss_flattened_name"] = ""
        row["map_approach"] = ""

        if not stmt or not path_raw:
            print(f"{tag} | [SKIPPED: missing statement or path]")
            output_rows.append(row)
            continue

        has_match = False

        # --- Approach 1: Description only ---
        if desc:
            query1 = f"{stmt} > {desc}"
            idx, score, name = try_match(query1, stmt, balance, tag)
            if idx is not None:
                has_match = True

                row["ofss_id"] = str(ofss_ids[idx])
                row["ofss_flattened_name"] = name.lower().replace("/", " > ")
                row["map_approach"] = "1"
                row['score'] = score
                print(f"{tag} | [mapped.1 ({score:.2f})] {query1} → {row['ofss_flattened_name']}")
            else:
                print(f"{tag} | [unmapped] {query1}")
        else:
            print(f"{tag} | [unmapped: no description]")

        # --- Approach 2: Taxonomy path ---
        if has_match == False:
            path = clean_path(path_raw)
            query2 = f"{stmt} > {path}"
            idx, score, name = try_match(query2, stmt, balance, tag)
            if idx is not None:
                row["ofss_id"] = str(ofss_ids[idx])
                row["ofss_flattened_name"] = name.lower().replace("/", " > ")
                row["map_approach"] = "2"
                row['score'] = score
                print(f"{tag} | [mapped.2 ({score:.2f})] {query2} → {row['ofss_flattened_name']}")

        output_rows.append(row)

# === SAVE TO CSV ===
fieldnames = list(output_rows[0].keys())
with open(OUTPUT_CSV, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_rows)


Processing rows: 3it [00:00, 13.09it/s]

AccountsNotesAndLoansReceivableNetCurrent | [mapped.1 (0.88)] Balance Sheet > accounts notes and loans receivable net current → balance sheet > assets > notes receivable - current
ReceivablesLongTermContractsOrPrograms | [mapped.1 (0.87)] Balance Sheet > receivables long term contracts or programs → balance sheet > assets > notes receivable - long term
AccountsReceivableGrossNoncurrent | [mapped.1 (0.81)] Balance Sheet > accounts receivable gross noncurrent → balance sheet > assets > accounts receivable


Processing rows: 5it [00:00, 11.95it/s]

AccountsReceivableNet | [mapped.1 (0.89)] Balance Sheet > accounts receivable net → balance sheet > assets > accounts receivable
AccountsReceivableNetCurrent | [mapped.1 (0.85)] Balance Sheet > accounts receivable net current → balance sheet > assets > notes receivable - current
AccountsReceivableNetNoncurrent | [mapped.1 (0.82)] Balance Sheet > accounts receivable net noncurrent → balance sheet > assets > accounts receivable


Processing rows: 9it [00:00, 13.64it/s]

...

ProceedsFromAdvanceForAndContributionInAidOfConstructionFinancingActivity | [mapped.1 (0.81)] Cash Flow Statement > proceeds from advance for and contribution in aid of construction financing activity → cash flow > financing activities > other financing activities
InterestExpenseMandatorilyRedeemableFinancialInstrumentLiabilityClassified | [mapped.1 (0.75)] Income Statement > interest expense mandatorily redeemable financial instrument liability classified → income statement > interest expense - operating
RedeemableNoncontrollingInterestEquityEliminationOfAccretionDeconsolidationOfSubsidiary | [mapped.1 (0.83)] Equity Statement > redeemable noncontrolling interest equity elimination of accretion deconsolidation of subsidiary → equity statement > redeemable noncontrolling interest adjustments
IncreaseDecreaseInIncentiveFeePayableCapitalGainBased | [mapped.1 (0.73)] Cash Flow Statement > increase decrease in incentive fee payable capital gain based → cash flow > operating activities > ch

Processing rows: 2912it [02:23, 21.03it/s]

IncreaseDecreaseInProfessionalFeePayable | [mapped.1 (0.74)] Cash Flow Statement > increase decrease in professional fee payable → cash flow > operating activities > changes in accounts payable
IncreaseDecreaseInAdministrativeFeePayable | [mapped.1 (0.76)] Cash Flow Statement > increase decrease in administrative fee payable → cash flow > operating activities > changes in accounts payable
IncentiveFeePayableIncomeBased | [mapped.1 (0.75)] Balance Sheet > incentive fee payable income based → balance sheet > liabilities > deferred income tax
OrganizationalAndOfferingCostsExpense | [mapped.1 (0.82)] Income Statement > organizational and offering costs expense → income statement > other operating expenses
OrganizationalCostExpense | [mapped.1 (0.86)] Income Statement > organizational cost expense → income statement > other operating expenses


Processing rows: 2914it [02:23, 20.29it/s]

IncreaseDecreaseInIncentiveFeePayableIncomeBased | [mapped.1 (0.74)] Cash Flow Statement > increase decrease in incentive fee payable income based → cash flow > operating activities > changes in accounts payable
ManagementAndIncentiveFeesPayable | [mapped.1 (0.77)] Balance Sheet > management and incentive fees payable → balance sheet > liabilities > accounts payable


In [ ]:
# import pandas as pd

# # Load existing CSV
# df = pd.read_csv(OUTPUT_CSV)




# # Identify invalid mappings
# invalid_mask = ~df.apply(is_valid_mapping, axis=1)
# invalid_entries = df[invalid_mask]

# # Display number of invalid entries
# print(f"Found {len(invalid_entries)} invalid entries.")

# # Clear mappings for invalid entries to allow manual review or reprocessing
# df.loc[invalid_mask, ["ofss_id", "ofss_flattened_name", "map_approach", "score"]] = ""

# # Save the corrected CSV
# df.to_csv("data/with_ofss_ids_corrected.csv", index=False)

# # Show sample of corrections made for verification
# invalid_entries.sample(min(10, len(invalid_entries)), random_state=42)[['tag', 'statement_type', 'description', 'ofss_flattened_name', 'score']]